In [3]:
import pandas as pd
import numpy as np
import chess

def board_to_array(fen):
    board = chess.Board(fen)
    piece_dict = {'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
                  'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6}
    array = np.zeros((8, 8), dtype=np.int8)
    for i in range(64):
        piece = board.piece_at(i)
        if piece:
            array[i // 8][i % 8] = piece_dict[piece.symbol()]
    return array

def move_to_index(move):
    # Extract just the 'from' and 'to' squares, ignoring promotion
    from_square = move[:2]
    to_square = move[2:4]
    return chess.SQUARE_NAMES.index(from_square) * 64 + chess.SQUARE_NAMES.index(to_square)

# Load the data
train_df = pd.read_csv('/content/drive/MyDrive/train-an-ai-to-play-chess/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/train-an-ai-to-play-chess/test.csv')


# Preprocess training data
X_train = np.array([board_to_array(board) for board in train_df['board']])
y_train_score = np.array(train_df['black_score'])
y_train_move = np.array([move_to_index(move) for move in train_df['best_move']])

# Preprocess test data
X_test = np.array([board_to_array(board) for board in test_df['board']])

# Reshape input data for the neural network
X_train = X_train.reshape((-1, 8, 8, 1))
X_test = X_test.reshape((-1, 8, 8, 1))

In [ ]:
# Install necessary libraries (optional, in case they're not already installed in Colab)
!pip install tensorflow chess pandas

# Import necessary libraries
import tensorflow as tf
import numpy as np
import chess
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, Reshape, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.mixed_precision import set_global_policy

# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Set up mixed precision for better GPU performance
set_global_policy('mixed_float16')


In [5]:
def residual_block(x, filters, kernel_size=3):
    y = Conv2D(filters, kernel_size, padding="same", kernel_regularizer=l2(1e-4))(x)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y = Conv2D(filters, kernel_size, padding="same", kernel_regularizer=l2(1e-4))(y)
    y = BatchNormalization()(y)
    return Add()([x, y])

def create_improved_model():
    board_input = Input(shape=(8, 8, 12))
    meta_input = Input(shape=(7,))

    # Initial convolutional layer
    x = Conv2D(256, 3, padding="same", kernel_regularizer=l2(1e-4))(board_input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Residual blocks
    for _ in range(19):
        x = residual_block(x, 256)

    # Policy head
    policy = Conv2D(256, 1, kernel_regularizer=l2(1e-4))(x)
    policy = BatchNormalization()(policy)
    policy = Activation("relu")(policy)
    policy = Flatten()(policy)
    policy = Dense(4096, activation="softmax", kernel_regularizer=l2(1e-4), name="policy")(policy)

    # Value head
    value = Conv2D(1, 1, kernel_regularizer=l2(1e-4))(x)
    value = BatchNormalization()(value)
    value = Activation("relu")(value)
    value = Flatten()(value)
    value = Dense(256, activation="relu", kernel_regularizer=l2(1e-4))(value)
    value = Concatenate()([value, meta_input])
    value = Dense(1, activation="tanh", kernel_regularizer=l2(1e-4), name="value")(value)

    # Create model
    model = Model(inputs=[board_input, meta_input], outputs=[policy, value])

    # Compile model with mixed precision and loss scale optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

    model.compile(
        optimizer=optimizer,
        loss={"policy": "categorical_crossentropy", "value": "mean_squared_error"},
        metrics={"policy": "accuracy", "value": "mae"}
    )

    return model


In [6]:
def board_to_planes(fen):
    board = chess.Board(fen)
    planes = np.zeros((8, 8, 12), dtype=np.float32)
    piece_dict = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    for i in range(64):
        piece = board.piece_at(i)
        if piece:
            planes[i // 8, i % 8, piece_dict[piece.symbol()]] = 1
    return planes

def get_meta_features(fen):
    board = chess.Board(fen)
    return np.array([
        int(board.has_kingside_castling_rights(chess.WHITE)),
        int(board.has_queenside_castling_rights(chess.WHITE)),
        int(board.has_kingside_castling_rights(chess.BLACK)),
        int(board.has_queenside_castling_rights(chess.BLACK)),
        int(board.has_legal_en_passant()),
        board.halfmove_clock / 100.0,
        board.fullmove_number / 100.0
    ], dtype=np.float32)


In [7]:
def data_generator(df, batch_size=32):
    def gen():
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]
            X_board = np.array([board_to_planes(fen) for fen in batch_df['board']])
            X_meta = np.array([get_meta_features(fen) for fen in batch_df['board']])

            if 'black_score' in batch_df.columns:
                y_value = np.array(batch_df['black_score'])
            else:
                y_value = np.zeros(len(batch_df))

            y_policy = np.ones((len(batch_df), 4096)) / 4096

            yield (
                (tf.convert_to_tensor(X_board, dtype=tf.float32),
                 tf.convert_to_tensor(X_meta, dtype=tf.float32)),
                (tf.convert_to_tensor(y_policy, dtype=tf.float32),
                 tf.convert_to_tensor(y_value, dtype=tf.float32))
            )

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            (tf.TensorSpec(shape=(None, 8, 8, 12), dtype=tf.float32),
             tf.TensorSpec(shape=(None, 7), dtype=tf.float32)),
            (tf.TensorSpec(shape=(None, 4096), dtype=tf.float32),
             tf.TensorSpec(shape=(None,), dtype=tf.float32))
        )
    )


In [ ]:
# Assume train_df and test_df are pandas DataFrames that you've already prepared.
# Replace these with the actual DataFrame variables you're using.
train_gen = data_generator(train_df, batch_size=64)
val_gen = data_generator(test_df, batch_size=64)

# Create and train the improved model
improved_model = create_improved_model()

# Train the model
history = improved_model.fit(
    train_gen,
    epochs=100,
    validation_data=val_gen
)

# Save the improved model
improved_model.save('improved_chess_ai_model.h5')


Epoch 1/100
    919/Unknown 148s 93ms/step - loss: 159975.3594 - policy_accuracy: 4.7429e-05 - value_mae: 274.9948

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


919/919 ━━━━━━━━━━━━━━━━━━━━ 160s 106ms/step - loss: 159979.3906 - policy_accuracy: 4.7396e-05 - value_mae: 274.9961 - val_loss: 10.5510 - val_policy_accuracy: 0.0000e+00 - val_value_mae: 0.9634
Epoch 2/100
919/919 ━━━━━━━━━━━━━━━━━━━━ 61s 66ms/step - loss: 159916.4062 - policy_accuracy: 0.0000e+00 - value_mae: 274.9314 - val_loss: 10.7788 - val_policy_accuracy: 0.0000e+00 - val_value_mae: 0.9789
Epoch 3/100
919/919 ━━━━━━━━━━━━━━━━━━━━ 63s 69ms/step - loss: 159911.5938 - policy_accuracy: 2.4431e-04 - value_mae: 274.9241 - val_loss: 11.0397 - val_policy_accuracy: 0.0000e+00 - val_value_mae: 0.9782
Epoch 4/100
919/919 ━━━━━━━━━━━━━━━━━━━━ 64s 70ms/step - loss: 159903.3906 - policy_accuracy: 0.0000e+00 - value_mae: 274.9164 - val_loss: 11.5471 - val_policy_accuracy: 0.0000e+00 - val_value_mae: 0.9927
Epoch 5/100
919/919 ━━━━━━━━━━━━━━━━━━━━ 64s 70ms/step - loss: 159888.7812 - policy_accuracy: 0.0130 - value_mae: 274.8957 - val_loss: 11.9470 - val_policy_accuracy: 0.0000e+00 - val_value_m

In [ ]:
# Function to play a game and export PGN
def play_game_with_improved_model(model, max_moves=50):
    board = chess.Board()
    move_count = 0

    # Track the moves in PGN format
    game_moves = []

    while not board.is_game_over() and move_count < max_moves:
        print(f"Move {move_count + 1}")
        print(board)

        if board.turn == chess.WHITE:
            move = predict_move_improved(model, board)
            try:
                board.push_uci(move)
            except chess.IllegalMoveError:
                print(f"AI attempted illegal move: {move}. Choosing random move.")
                legal_moves = list(board.legal_moves)
                if legal_moves:
                    board.push(np.random.choice(legal_moves))
                else:
                    print("No legal moves available. Game over.")
                    break
        else:
            # Simulate black's random move
            legal_moves = list(board.legal_moves)
            if legal_moves:
                board.push(np.random.choice(legal_moves))
            else:
                print("No legal moves available. Game over.")
                break

        # Add the move to the game_moves list in PGN format
        game_moves.append(board.san(board.peek()))

        move_count += 1
        print("\n")

    print(board)
    print("Game over")

    # Export the game as PGN
    game_pgn = board.board_fen() + "\n\n"
    for i, move in enumerate(game_moves):
        if i % 2 == 0:
            game_pgn += f"{(i // 2) + 1}. {move} "
        else:
            game_pgn += f"{move} "

    print("Result:", board.result())
    print("\nPGN Format:")
    print(game_pgn)

    return game_pgn

# Play the game and get the PGN
pgn_output = play_game_with_improved_model(improved_model)

# Save the PGN to a file
with open('chess_game.pgn', 'w') as f:
    f.write(pgn_output)
